# Projeto - Motor de Recomendação

**Objetivo:** desenvolver um sistema de recomendação híbrido para e-commerce que combine:
- similaridade entre itens (cosine on item vectors / TF-IDF),
- regras de associação (apriori / co-ocorrência),
- segmentação de clientes (KMeans) para personalização por cluster.

## Fluxograma do Processo

flowchart TD
  A[1. Objetivo de Negócio] --> B[2. Mapeamento de Dados]
  B --> C[3. Ingestão (CSV -> MySQL)]
  C --> D[4. EDA SQL (sanity checks, top products, RFM, co-ocorrência)]
  D --> E[5. Export p/ Python (pandas)]
  E --> F[6. Limpeza & Preprocessing]
  F --> G[7. Feature Engineering]
  G --> H[8. Modelagem ML (Hybrid Recommender)]
  H --> I[9. Avaliação & Interpretação]
  I --> J[10. Deploy (AWS RDS, SageMaker, API Gateway)]
  J --> K[11. Monitoramento & Entrega (report + notebook + quarto)]


## Definição de Objetivos

**Objetivo de negócio:** Aumentar ticket médio e conversão através de recomendações personalizadas.
**KPIs:** aumento do ticket médio (AOV), taxa de conversão, CTR de recomendações, receita incremental.
**Requisitos de dados:** histórico de transações (linha por item), catálogo de produtos, visualizações, atributos do cliente (idade, cidade), timestamps.


## Mapeamento do Fluxo de Dados e Processos de Negócio

**Fonte dos dados:** dados sintéticos gerados em Python (Faker) para simular comportamento real.  
**Pipeline de ingestão:** CSVs → `LOAD DATA LOCAL INFILE` → MySQL RDS/local.  
**Observações:** No ambiente real desejaríamos order_id por pedido; esse dataset usa transaction rows por item. Considerar adição de session_id e device_type (já existe em product_views).


## EDA - Exploratory Data Analysis

In [1]:
from getpass import getpass
import os
import pandas as pd
from sqlalchemy import create_engine, text
import matplotlib.pyplot as plt

In [ ]:
DB_USER = "root"
DB_HOST = "127.0.0.1"
DB_PORT = 3306
DB_NAME = "ecommerce_db"
DB_PASS = getpass("DB password: ")

CONN_STR = f"mysql+pymysql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine = create_engine(CONN_STR, echo=False)

In [ ]:
def run_sql_pure(sql: str, params: dict = None):
    """
    Executa SQL puro (string) e retorna um pandas.DataFrame.
    Exibe a query (útil para portfólio) e retorna o DataFrame.
    """
    print("---- Executing SQL ----")
    print(sql.strip())
    print("-----------------------")
    df = pd.read_sql_query(sql=text(sql), con=engine, params=params)
    display(df.head(10))
    return df

# Configuração do matplotlib (opcional)
plt.rcParams['figure.figsize'] = (10,5)
plt.rcParams['grid.linestyle'] = '--'

**Sanity checks — contagens por tabela**

In [ ]:
%%sql
SELECT 
  COUNT(*) AS total_rows,
  COUNT(DISTINCT transaction_id) AS distinct_transaction_id,
  COUNT(DISTINCT CONCAT(customer_id,'-',DATE(transaction_date))) AS distinct_cust_date
FROM transactions;

In [ ]:
sql = """
SELECT 
  (SELECT COUNT(*) FROM customers) AS n_customers,
  (SELECT COUNT(*) FROM products)  AS n_products,
  (SELECT COUNT(*) FROM transactions) AS n_transactions,
  (SELECT COUNT(*) FROM product_views) AS n_views;
"""
df = run_sql_pure(sql)
vals = df.iloc[0].to_dict()
names = list(vals.keys()); counts = list(vals.values())


**Checar valores ausentes por tabela**

In [ ]:
# lista de tabelas a verificar (ajuste se tiver nomes diferentes)
tables = ['customers', 'products', 'transactions', 'product_views']

results = []
for t in tables:
    # pegar colunas da tabela via information_schema
    q_cols = f"""
    SELECT COLUMN_NAME
    FROM INFORMATION_SCHEMA.COLUMNS
    WHERE TABLE_SCHEMA = '{DB_NAME}'
      AND TABLE_NAME = '{t}'
    ORDER BY ORDINAL_POSITION;
    """
    cols_df = pd.read_sql_query(sql=text(q_cols), con=engine)
    cols = cols_df['COLUMN_NAME'].tolist()
    num_cols = len(cols)

    # contar linhas
    q_rows = f"SELECT COUNT(*) AS n_rows FROM {t};"
    n_rows = pd.read_sql_query(sql=text(q_rows), con=engine).iloc[0,0]

    if n_rows == 0 or num_cols == 0:
        null_count = 0
    else:
        # construir expressão que soma "IS NULL" por coluna
        # ex: SUM(col1 IS NULL) + SUM(col2 IS NULL) + ...
        null_expr = " + ".join([f"SUM(`{c}` IS NULL)" for c in cols])
        q_nulls = f"SELECT {null_expr} AS null_count FROM {t};"
        null_count = pd.read_sql_query(sql=text(q_nulls), con=engine).iloc[0,0]

    total_cells = int(n_rows) * int(num_cols)
    pct_null = (null_count / total_cells * 100) if total_cells > 0 else 0.0

    results.append({
        "table": t,
        "n_rows": int(n_rows),
        "n_cols": int(num_cols),
        "total_cells": int(total_cells),
        "null_count": int(null_count),
        "pct_null": round(pct_null, 4)
    })

df_missing_tables = pd.DataFrame(results)
display(df_missing_tables)

**Duplicados e chaves (checar chave primária única) — transactions**

In [ ]:
%%sql
SELECT 
  COUNT(*) AS total_rows,
  COUNT(DISTINCT transaction_id) AS distinct_transaction_id,
  COUNT(DISTINCT CONCAT(customer_id,'-',DATE(transaction_date))) AS distinct_cust_date
FROM transactions;

In [ ]:
sql = """
SELECT 
  COUNT(*) AS total_rows,
  COUNT(DISTINCT transaction_id) AS distinct_transaction_id,
  COUNT(DISTINCT CONCAT(customer_id,'-',DATE(transaction_date))) AS distinct_cust_date
FROM transactions;
"""
df = run_sql_pure(sql)
print(df.to_string(index=False))

**Distribuição de preços — produtos (histograma)**

In [ ]:
%%sql
SELECT price FROM products WHERE price IS NOT NULL LIMIT 10000;

In [ ]:
sql = "SELECT price FROM products WHERE price IS NOT NULL;"
df_price = run_sql_pure(sql)
# converter coluna price se necessário
df_price['price'] = pd.to_numeric(df_price['price'], errors='coerce')
plt.hist(df_price['price'].dropna(), bins=30)
plt.title("Distribuição de preços dos produtos")
plt.xlabel("Preço")
plt.ylabel("Frequência")
plt.show()

**Top produtos por quantidade e por receita (SQL puro)**

In [ ]:
%%sql
SELECT p.product_id, p.name, p.category, SUM(t.quantity) AS total_qty
FROM transactions t
JOIN products p ON t.product_id = p.product_id
GROUP BY p.product_id
ORDER BY total_qty DESC
LIMIT 20;

In [ ]:
sql = """
SELECT p.product_id, p.name,
       SUM(t.quantity) AS total_qty,
       SUM(t.total_value) AS total_revenue
FROM transactions t
JOIN products p ON t.product_id = p.product_id
GROUP BY p.product_id
ORDER BY total_qty DESC
LIMIT 15;
"""
df_top = run_sql_pure(sql)
ax = df_top.plot.bar(x='name', y='total_qty', legend=False)
plt.xticks(rotation=70, fontsize=8)
plt.title("Top 15 produtos por quantidade vendida")
plt.ylabel("Quantidade")
plt.show()

# revenue bar (same products)
df_top.plot.bar(x='name', y='total_revenue')
plt.xticks(rotation=70, fontsize=8)
plt.title("Top 15 produtos por receita (mesmos produtos)")
plt.ylabel("Receita")
plt.show()

**Vendas por mês (série temporal)**

In [ ]:
%%sql
SELECT DATE_FORMAT(transaction_date, '%Y-%m') AS ym,
       COUNT(*) AS n_items,
       SUM(total_value) AS revenue
FROM transactions
GROUP BY ym
ORDER BY ym;

In [ ]:
sql = """
SELECT DATE_FORMAT(transaction_date, '%Y-%m') AS ym,
       COUNT(*) AS n_items,
       SUM(total_value) AS revenue
FROM transactions
GROUP BY ym
ORDER BY ym;
"""
df_ts = run_sql_pure(sql)
df_ts['ym'] = pd.to_datetime(df_ts['ym'] + '-01')
plt.plot(df_ts['ym'], df_ts['revenue'])
plt.title("Receita por mês")
plt.xlabel("Mês")
plt.ylabel("Receita")
plt.grid(True)
plt.show()

**RFM — tabela e histogramas (SQL + Python)**

In [ ]:
%%sql
WITH last AS (
  SELECT customer_id,
         DATEDIFF(CURDATE(), MAX(DATE(transaction_date))) AS recency_days,
         COUNT(*) AS frequency,
         SUM(total_value) AS monetary
  FROM transactions
  GROUP BY customer_id
)
SELECT * FROM last;

In [ ]:
sql = """
WITH last AS (
  SELECT customer_id,
         DATEDIFF(CURDATE(), MAX(DATE(transaction_date))) AS recency_days,
         COUNT(*) AS frequency,
         SUM(total_value) AS monetary
  FROM transactions
  GROUP BY customer_id
)
SELECT * FROM last;
"""
df_rfm = run_sql_pure(sql)
# Histograms
fig, axes = plt.subplots(1,3, figsize=(15,4))
axes[0].hist(df_rfm['recency_days'].dropna(), bins=20); axes[0].set_title('Recency (dias)')
axes[1].hist(df_rfm['frequency'].dropna(), bins=20); axes[1].set_title('Frequency')
axes[2].hist(df_rfm['monetary'].dropna(), bins=20); axes[2].set_title('Monetary')
plt.show()

# scatter recency x monetary
plt.scatter(df_rfm['recency_days'], df_rfm['monetary'])
plt.xlabel('Recency (dias)'); plt.ylabel('Monetary (total_spent)')
plt.title('Recency x Monetary (clientes)')
plt.grid(True)
plt.show()

**Distribuição de compras por cliente (histograma) — SQL + Python**

%%sql
SELECT customer_id, COUNT(*) AS purchases
FROM transactions
GROUP BY customer_id
ORDER BY purchases DESC
LIMIT 100;

In [ ]:
sql = """
SELECT customer_id, COUNT(*) AS purchases
FROM transactions
GROUP BY customer_id;
"""
df_purchases = run_sql_pure(sql)
plt.hist(df_purchases['purchases'], bins=30)
plt.title("Distribuição de número de compras por cliente")
plt.xlabel("Número de compras")
plt.ylabel("Clientes")
plt.show()

**Conversão views -> buys por produto (scatter/identify outliers)**

In [ ]:
%%sql
SELECT p.product_id, p.name,
  COALESCE(v.views,0) AS views,
  COALESCE(b.buys,0)  AS buys,
  ROUND(COALESCE(b.buys,0) / NULLIF(COALESCE(v.views,0),0) * 100,2) AS conversion_pct
FROM products p
LEFT JOIN (
  SELECT product_id, COUNT(*) AS views
  FROM product_views
  GROUP BY product_id
) v ON p.product_id = v.product_id
LEFT JOIN (
  SELECT product_id, COUNT(*) AS buys
  FROM transactions
  GROUP BY product_id
) b ON p.product_id = b.product_id
ORDER BY conversion_pct DESC
LIMIT 200;

In [ ]:
sql = """
SELECT p.product_id, p.name,
  COALESCE(v.views,0) AS views,
  COALESCE(b.buys,0)  AS buys,
  ROUND(COALESCE(b.buys,0) / NULLIF(COALESCE(v.views,0),0) * 100,2) AS conversion_pct
FROM products p
LEFT JOIN (
  SELECT product_id, COUNT(*) AS views
  FROM product_views
  GROUP BY product_id
) v ON p.product_id = v.product_id
LEFT JOIN (
  SELECT product_id, COUNT(*) AS buys
  FROM transactions
  GROUP BY product_id
) b ON p.product_id = b.product_id;
"""
df_conv = run_sql_pure(sql)
plt.scatter(df_conv['views'], df_conv['buys'])
plt.xlabel('Views'); plt.ylabel('Buys')
plt.title('Views x Buys (produtos)')
plt.grid(True)
plt.show()

# print top products with unusually high conversion (>80%)
high_conv = df_conv[df_conv['conversion_pct'] > 80].sort_values('conversion_pct', ascending=False)
print("Produtos com conversão > 80% (exemplo):")
display(high_conv[['product_id','name','views','buys','conversion_pct']].head(10))

**Vendas por categoria (barra)**

In [ ]:
%%sql
SELECT p.category, COUNT(*) AS n_items, SUM(t.total_value) AS revenue
FROM transactions t
JOIN products p ON t.product_id = p.product_id
GROUP BY p.category
ORDER BY revenue DESC;

In [ ]:
sql = """
SELECT p.category, COUNT(*) AS n_items, SUM(t.total_value) AS revenue
FROM transactions t
JOIN products p ON t.product_id = p.product_id
GROUP BY p.category
ORDER BY revenue DESC;
"""
df_cat = run_sql_pure(sql)
df_cat.plot.bar(x='category', y='revenue')
plt.title("Receita por categoria")
plt.xticks(rotation=60)
plt.ylabel("Receita")
plt.show()

**AOV (Average Order Value) — aproximação por customer_date**

In [ ]:
%%sql
SELECT customer_id, DATE(transaction_date) AS order_date, SUM(total_value) AS order_value
FROM transactions
GROUP BY customer_id, DATE(transaction_date);


In [ ]:
sql = """
SELECT customer_id, DATE(transaction_date) AS order_date, SUM(total_value) AS order_value
FROM transactions
GROUP BY customer_id, DATE(transaction_date);
"""
df_orders = run_sql_pure(sql)
aov = df_orders['order_value'].mean()
print(f"AOV (média de valor por pedido - aproximação): R$ {aov:.2f}")
plt.hist(df_orders['order_value'], bins=30)
plt.title("Distribuição do valor por pedido (aprox.)")
plt.xlabel("Order value")
plt.ylabel("Contagem")
plt.show()


**Matriz cliente×produto densidade (sparsity) — para decidir estratégia ML**

In [ ]:
%%sql
SELECT (SELECT COUNT(DISTINCT customer_id) FROM transactions) AS n_customers_active,
       (SELECT COUNT(DISTINCT product_id) FROM transactions) AS n_products_sold,
       (SELECT COUNT(*) FROM transactions) AS n_transactions;


In [ ]:
sql = """
SELECT (SELECT COUNT(DISTINCT customer_id) FROM transactions) AS n_customers_active,
       (SELECT COUNT(DISTINCT product_id) FROM transactions) AS n_products_sold,
       (SELECT COUNT(*) FROM transactions) AS n_transactions;
"""
df_dim = run_sql_pure(sql)
n_customers = int(df_dim['n_customers_active'].iloc[0])
n_products = int(df_dim['n_products_sold'].iloc[0])
n_trans = int(df_dim['n_transactions'].iloc[0])
possible = n_customers * n_products
sparsity = 1 - (n_trans / possible)
print(f"Clientes: {n_customers}, Produtos: {n_products}, Transações: {n_trans}")
print(f"Sparsity (aprox): {sparsity:.6f}")

## Insights

* O portfólio de produtos tem ampla faixa de preços, mas as vendas e a receita são **fortemente concentradas em Eletrônicos**, indicando padrões claros de preferência.
* A maioria dos clientes realiza entre **7 e 12 compras**, com grande variabilidade em gasto total — ideal para segmentação via **RFM e clustering**.
* A receita mensal mostra **crescimento consistente**, sugerindo comportamento sazonal que pode ser explorado futuramente.
* Alguns produtos apresentam **altas taxas de conversão (views → compras)**, tornando-se bons candidatos a recomendações de destaque (*best-sellers* e *mais vistos*).
* A matriz cliente × produto é **altamente esparsa (97%+)**, confirmando que os modelos mais adequados são:
  **Item-based Filtering**, **Content-based**, **Association Rules** e abordagem **Híbrida**.

Os padrões observados suportam a construção de um sistema de recomendação híbrido, combinando similaridade entre itens, regras de coocorrência, conteúdo dos produtos e segmentação de clientes.

# Parte COMEÇA NOTEBOOK DE ESTUDO (APAGAR DEPOIS)


# 1 — Visão geral (o problema que resolvemos)

Um sistema de recomendação tenta prever **o que um usuário vai gostar/comprar** a seguir, baseado em dados passados (transações, visualizações, perfil). Em e-commerce o objetivo prático é aumentar **conversão**, **ticket médio (AOV)**, **retenção** e **engajamento**.

Analogia: imagine um vendedor experiente numa loja — ao ver o carrinho do cliente e ouvir suas preferências, ele recomenda itens que normalmente complementam a compra. Nosso sistema faz o mesmo, mas em escala, usando padrões estatísticos e de similaridade.

---

# 2 — Tipos de recomendadores (teoria + analogias)

## 2.1 Item-based Collaborative Filtering (Item-Item CF)

**O conceito:** recomenda itens semelhantes aos que o usuário já comprou, medindo similaridade entre itens a partir do comportamento dos usuários (quem comprou A também comprou B).

**Matemática (cosine similarity):**
Se cada item j tem um vetor de ocorrência (v_j) onde a entrada (v_{j,u}) é 1 se usuário (u) comprou j (ou outra medida como quantidade), então a similaridade entre itens i e j por **cosine** é:

[
\text{sim}_{\cos}(i,j) = \frac{v_i \cdot v_j}{|v_i| , |v_j|}
]

Interpretação: quanto maior este valor, mais os mesmos usuários compraram ambos os itens.

**Por que funciona bem em e-commerce:** é robusto frente a sparsity relativa (few interactions per user) e rápido no runtime se pré-computamos as similaridades (indexação). Além disso, recomenda itens que efetivamente foram comprados juntos por outros usuários (bom para conversão).

**Analogia:** “Clientes que compraram este livro também compraram aquele outro” — como uma prateleira que organiza itens por quem os pegou.

**Variações práticas:** usar co-occurrence counts, normalização por popularidade (para não favorecer item muito popular), ou aplicar shrinkage (suavização) para pares com poucas co-ocorrências.

---

## 2.2 Content-Based Filtering (TF-IDF + cosine)

**O conceito:** descrevemos produtos por atributos (nome, categoria, descrição) e recomendamos itens semanticamente próximos ao que o usuário já consumiu.

**TF-IDF + cosine:** representamos cada item como vetor TF-IDF sobre termos (nome+categoria). A similaridade entre item i e j novamente pode ser cosine entre esses vetores textuais.

**Por que é útil:** resolve **cold-start de items** (novo produto sem compras) e complementa CF quando não existe histórico. Também captura afinidade por atributos (marca, características).

**Analogia:** o vendedor lê a etiqueta e fala “se gosta deste, pode gostar daquele com etiquetas semelhantes”.

---

## 2.3 Association Rules (Apriori / co-occurrence / lift)

**O conceito:** extrair regras do tipo (A \Rightarrow B) — se o cliente comprou A, a probabilidade de comprar B aumenta. Usamos suporte, confiança e lift:

* **Support(A)** = fracção de transações que contêm A.
* **Support(A ∧ B)** = fracção de transações que contêm A e B.
* **Confidence(A → B)** = Support(A ∧ B) / Support(A). (probabilidade condicional)
* **Lift(A → B)** = Confidence(A→B) / Support(B) = Support(A∧B) / (Support(A) Support(B))

**Interpretação:** lift > 1 indica que A e B ocorrem juntos mais frequentemente do que por acaso — boa regra de cross-sell.

**Por que usar:** regras são simples, interpretáveis e ótimas para “produtos comprados juntos” (compre junto / up-sell).

**Analogia:** cesta de compras — se alguém pega pão e manteiga repetidamente, sugere-se sempre a manteiga ao lado do pão.

---

# 3 — Feedback: explícito vs implícito; binarização e confiança

No mundo real de e-commerce geralmente não temos ratings (explícitos); temos **feedback implícito**: compra, view, clique, tempo de sessão. Esse feedback é ruidoso e ambíguo:

* Compra é forte sinal positivo.
* View é sinal fraco/interest.
* Ausência de interação é *indeterminação* (o usuário pode não ter visto o produto).

**Tratamento prático (implicito):**

* Binarizar: (r_{u,i} = 1) se comprou, 0 caso contrário — simples e funciona bem para top-N.
* Ponderar por confiança: usar quantidade, frequência ou transformar views em score fraco (ex.: view=0.1, compra=1.0).
* Modelo de confiança (Hu, Koren & Volinsky — 2008): usa matriz (c_{ui} = 1 + \alpha r_{ui}) como "confiança" em algoritmos ALS para implicit data.

**Analogia:** comprar é como um voto forte; ver é levantar a mão discretamente.

---

# 4 — Sparse data (sparsity) e implicações práticas

**O que é sparsity:** proporção de células zero na matriz usuário×produto. Alta sparsity (>= 95%) é a realidade em e-commerce.

**Consequências:**

* Modelos que exigem muitos dados por usuário (fatoração pura, redes neurais pesadas) ficam menos robustos.
* Abordagens que exploram conteúdo e co-ocorrência (item-item, association rules) tendem a performar melhor.
* Importante: usar regularização, shrinkage e smoothing para evitar overfitting em pares raros.

**Analogia:** é como ter poucas anotações por cliente — o vendedor precisa usar sinais gerais (popularidade e características do produto) e padrões entre clientes para inferir recomendações.

---

# 5 — Medidas de similaridade — quando usar qual

* **Cosine similarity:** recomendado para vetores TF-IDF e vetores item×user binários. Simples, escala bem.
* **Jaccard index:** bom para binários (interseção sobre união) quando queremos penalizar popularidade:
  (\text{J}(A,B)=\frac{|U_A\cap U_B|}{|U_A\cup U_B|}).
* **Adjusted Cosine / Pearson:** ajusta por comportamento médio do usuário — útil para ratings.
* **Pointwise Mutual Information (PMI):** captura associação estatística ponto a ponto; bom para co-occurrence raros.
* **Co-occurrence raw counts:** bom indicador bruto; porém favorece itens populares.

**Decisão prática:** para item-item CF e TF-IDF, **cosine** é padrão; podemos usar **shrinkage**:
[
\text{sim}*{\text{shrunk}}(i,j) = \frac{|C*{ij}|}{|C_{ij}| + \lambda} \cdot \text{sim}*{\text{raw}}(i,j)
]
onde (|C*{ij}|) = co-occurrence count, (\lambda) = parâmetro de suavização.

Analogia: dar mais confiança a pares com mais evidência.

---

# 6 — Híbridos — como combinar sinais (fórmulas práticas)

Queremos combinar sinais distintos: item-item CF, content similarity, association rules, e um componente de popularidade. Uma fórmula simples:

[
\text{score}*{u,i} = w_1 \cdot \max*{j \in H_u} \text{sim}*{item}(i,j) ;+; w_2 \cdot \text{sim}*{content}(i, H_u^{\text{centroid}}) ;+; w_3 \cdot \text{assoc_score}(i,|,H_u) ;+; w_4 \cdot \text{popularity}_{i}
]

* (H_u): histórico de itens do usuário (u).
* (\max_{j\in H_u}) é o esquema de *max pooling* (recomenda baseada no item mais similar do histórico) — simples e eficaz. Alternativa: média ponderada.
* (\text{assoc_score}) pode ser confiança/lift somados para itens em (H_u).
* pesos (w_k) são ajustáveis (grid search ou aprendizado).

**Normalização:** cada componente precisa estar na mesma escala (z-score, min-max, ou softmax) antes de combinar, para evitar que um domine apenas por escala.

**Analogia:** o vendedor junta recomendações do colega (item-item), do catálogo (conteúdo), e das caixas “frequentemente comprados juntos” (association rules), e pondera conforme contexto.

---

# 7 — Avaliação offline — como saber que funciona

Objetivo: medir qualidade de ranking para top-N (ex.: top-10). Estratégia:

* **Split temporal**: treinar com transações até T, testar nas transações de T+1 (evita vazamento).
* **Holdout per user**: remover um item por usuário para teste (leave-one-out) e ver se modelo recupera.
* **Métricas**:

  * **Precision@K**: proporção de itens relevantes em top-K.
  * **Recall@K**: proporção dos itens relevantes recuperados.
  * **MAP@K** (Mean Average Precision) — penaliza ordem.
  * **NDCG@K** — penaliza posição (mais sensível à ordem).
  * **Coverage**: % de itens que podem ser recomendados.
  * **Novelty / Serendipity**: quão inesperadas as recomendações são (negócio pode querer tradeoff).
  * **Hit rate**: se test item está em top-K.

**Observação prática:** para implicit feedback, usar ranking metrics (precision@K, NDCG) é o mais apropriado.

**Analogia:** é como avaliar um vendedor por quantas frases de venda dele resultam em uma compra efetiva.

---

# 8 — Aspectos de produção / deploy e valor de negócio

**Pipeline realista (E2E):**

1. Preprocess: gerar item_embeddings (TF-IDF), item-item similarity matrix, association rules table.
2. Offline training/tuning com validação temporal.
3. Build recommender endpoint: função `recommend(user_id, k)` que acessa:

   * histórico do user (do RDS),
   * similarities (cache/memcached),
   * regras precomputadas,
   * ranking final e filtros (estoque, business rules).
4. Serving: SageMaker endpoint / microservice (FastAPI) + API Gateway.
5. Monitoramento: CTR das recomendações, conversão, AOV incremental, cobertura, latency.

**KPIs de negócio:** aumento % no AOV, uplift de receita por usuário, CTR de recomendações.

**Analogia:** imaginar um assistente digital que consulta o histórico do cliente, um índice de similaridade (como um catálogo inteligente), e regras pré-definidas para sugerir complementos — tudo em <100ms.

---

# 9 — Decisões técnicas que iremos justificar (e como justificaremos)

Quando tomarmos cada decisão, vou explicar com dados do seu projeto:

* Por que usar **cosine** vs **Jaccard** (ex.: densidade, dados binários ou ponderados).
* Quando **binarizar** (compra=1) vs usar quantidade/valor (se isso melhora métricas).
* Como ajustar **pesos** (w_1,w_2,w_3) do híbrido (grid search ou small supervised model).
* Se aplicamos **shrinkage** em similaridade (quando pares têm baixa contagem).
* Como transformar **views → signal** (peso para view < compra) ou ignorar.
* Estratégia de **cold start** para novos usuários (usar conteúdo + popularidade) e novos produtos (content-based).

Para cada escolha eu apresentarei:

* experimento/justificativa (resultado em dados),
* impacto esperado no KPIs (intuitivo e quantificável),
* trade-offs.

---

# 10 — Pequenas analogias finais para fixar

* **Item-item CF** é como perguntar “o que compradores semelhantes levaram?”. (Bom quando existem muitos compradores.)
* **Content-based** é como comparar etiquetas/descrições — ótimo para novos produtos.
* **Association rules** é como a cesta de compras: “sempre que compram X, levam Y”.
* **Híbrido** é o vendedor que combina experiência, catálogo e histórico de compras para oferecer a melhor sugestão.

---

# Conclusão / próximo passo prático

Agora que você tem a base teórica, podemos **partir para a implementação** passo-a-passo — começando por preparar as estruturas:

1. extrair a matriz cliente×produto (binária/ponderada),
2. construir item-item co-occurrence e similarity (com shrinkage),
3. gerar TF-IDF dos textos do produto,
4. extrair regras de associação com suporte/ lift,
5. montar pipeline de scoring híbrido e avaliar offline.

Quando você me disser qual componente preferir começar (Item-Item, Content, Association ou já direto híbrido), eu te mostro o **código passo a passo** e, para cada escolha, a explicação técnica + alternativa e qual KPI impacta.

Quer começar pela **Item-Item CF** (recomendado) ou prefere **Content-based** primeiro?
